In [17]:
import os
import glob
import psycopg2
import pandas as pd
from sql_queries import *
import pandas.io.sql as sqlio

from matplotlib import pyplot

import warnings
warnings.filterwarnings('ignore')

In [4]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
cur = conn.cursor()

In [7]:
df = sqlio.read_sql_query("SELECT * FROM songplays", conn)

## Doing a little exploratody analysis

In [11]:
print(f'The shape of the dataset is: {df.shape}')

The shape of the dataset is: (6820, 9)


In [14]:
print(f'The dataset contains this amount of null values per columns: {df.isnull().sum()}')

The dataset contains this amount of null values per columns: songplay_id       0
start_time        0
user_id           0
level             0
song_id        6819
artist_id      6819
session_id        0
location          0
user_agent        0
dtype: int64


#### Let's check the values of the columns to see if we find out something cool

In [38]:
df['start_time'].sample(5)

5704   1970-01-01 00:25:41.607557796
4299   1970-01-01 00:25:42.846220796
5880   1970-01-01 00:25:42.373636796
1813   1970-01-01 00:25:41.254670796
1726   1970-01-01 00:25:42.315784796
Name: start_time, dtype: datetime64[ns]

We can see the start_time is still in ms time, so let's convert it into a human friendly format like a normal timestamp and sample some values

In [40]:
df['start_time'] = pd.to_datetime(df['start_time'])
df['start_time'].sample(5)

616    1970-01-01 00:25:43.003488796
132    1970-01-01 00:25:43.574980796
4358   1970-01-01 00:25:42.890195796
4733   1970-01-01 00:25:41.429782796
4900   1970-01-01 00:25:41.461788796
Name: start_time, dtype: datetime64[ns]

In [46]:
print(f"The date in which songs were streamed appear to be {df['start_time'].dt.date.value_counts()}")

The date in which songs were streamed appear to be 1970-01-01    6820
Name: start_time, dtype: int64


I wanted to make a time series of the songs listened but the only records that appear to be different are the minutes and the seconds, so I'm not going to enjoy taking any insights from it.

In [47]:
df['level'].value_counts()

paid    5591
free    1229
Name: level, dtype: int64

This column can be a boolean, I would actually like it more if it were to be a boolean column because I had no idea what that column meant when I started the project. Maybe a is_paid or is_premium or whatever

We will not be looking into the IDs columns since we know they contain only unique values and a lot of nulls. 

In [50]:
df['location'].value_counts().sample(6)

Portland-South Portland, ME          665
London, KY                             5
Augusta-Richmond County, GA-SC       140
Atlanta-Sandy Springs-Roswell, GA    456
Salt Lake City, UT                     4
Santa Rosa, CA                        20
Name: location, dtype: int64

In [57]:
new = df['location'].str.rsplit(',', n = 1, expand = True) 

df["city"]= new[0]
df["state_code"]= new[1]

In [58]:
df["city"].value_counts().sample(7)

Yuba City                        27
La Crosse-Onalaska               45
New York-Newark-Jersey City     262
Richmond                         13
Eureka-Arcata-Fortuna            16
Klamath Falls                    20
Portland-Vancouver-Hillsboro      7
Name: city, dtype: int64

In [60]:
df['state_code'].value_counts().head(7)

 CA          1572
 ME           665
 MI           636
 IL-IN-WI     475
 GA           456
 IA           407
 AZ           348
Name: state_code, dtype: int64

Now we know a lot of the locations are streamed from California, this operation was useful to separate the string into two columns for us to have this valuable state code.

In [37]:
df[df['user_agent'].str.contains('Mozilla')]['user_agent'].shape

(6820,)

Every song was streamed via Mozilla user agent, I really wanted there to be more

#### I will eventually come back to this EDA after I look up some online notebooks to see if I can still do something interesting. Overall, very nice project